# 패키지 로딩

In [2]:
import numpy as np
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

# 데이터 로딩

## 서울시 구별 남녀 인구수
- 출처: 통계청 행정구역(시군구)별 성별 인구수
- 데이터 분류: 월별 데이터
- 기간: 2014년 3월 ~ 2024년 2월 (120개월, 10년치)


In [3]:
df1 = pd.read_csv("./data/서울시 구단위 남녀 인구수.csv", header=[0, 1], encoding="cp949")

## 서울시 연령대별 남녀 인구수
- 출처: 통계청 시군구 성 연령 5세 별 주민등록연앙인구
- 데이터 분류: 연간 데이터
- 기간: 2014년 ~ 2023년 (10년치)

### 남자 인구수

In [4]:
df2 = pd.read_csv("./data/서울시 구단위 연령별 인구수_남.csv", index_col=[0, 1],encoding="cp949")

### 여자 인구수

In [5]:
df3 = pd.read_csv("./data/서울시 구단위 연령별 인구수_여.csv", index_col=[0, 1],encoding="cp949")

## 주민등록인구(내국인 각 세별/구별)
- 출처: 서울 열린데이터 광장
- 데이터 분류: 분기별

In [6]:
# 2023년 4분기 데이터
df4 = pd.read_excel("./data/주민등록인구(내국인 각 세별_구별)_2023년 4분기.xlsx", index_col = 0)
df4.index.name = None

## 세대원수별 세대수
- 출처: 서울 열린데이터 광장
- 데이터 분류: 월별

In [7]:
# 2023년 데이터
df5 = pd.read_excel("./data/2023년 세대원수별 세대수.xlsx", index_col=0)

## 주민등록인구(월별)
- 출처: 서울 열린데이터 광장
- 데이터 분류: 월별

In [ ]:
# 2023년 데이터
df6 = pd.read_excel("./data/2023년 주민등록인구.xlsx", index_col=0, header=[0, 1])

## 서울시 자치구 (목적별) CCTV 설치현황
- 출처: 서울 열린데이터 광장

In [88]:
df7 = pd.read_excel("./data/서울시 자치구 (목적별) CCTV 설치현황_231231.xlsx", index_col=0, header=[0, 1])

# 데이터 전처리

## 서울시 구단위 남녀 인구수

- 시점 pd.datetime으로 변경

In [9]:
# 데이터 타입 확인: float64
df1[("시점", "시점")].info()

<class 'pandas.core.series.Series'>
RangeIndex: 120 entries, 0 to 119
Series name: ('시점', '시점')
Non-Null Count  Dtype  
--------------  -----  
120 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


In [10]:
# float -> stirng -> pd.to_datetime()
df1[("시점", "시점")] = df1[("시점", "시점")].apply(lambda x: pd.to_datetime(str(x)))

df1 = df1.set_index(df1[("시점", "시점")]) 
df1 = df1.drop(('시점', '시점'), axis = 1)

- index 이름 초기화
- Multi columns 이름 설정

In [11]:
df1.index.name = None
df1.columns.names = ["구", "성별"]

- 시기별 서울시 전체 인구 계산

In [13]:
colList = list(df1.columns.get_level_values("구").unique())

print(len(colList))

25


In [14]:
df1_sum = pd.DataFrame()

for column in colList:
    temp = df1[column]
    temp[f"{column} 합계"] = temp.sum(axis = 1)
    
    df1_sum = pd.concat([df1_sum, temp[[f"{column} 합계"]]], axis = 1)

In [15]:
df1_sum["서울시"] = df1_sum.sum(axis = 1)

In [16]:
df1_prop = df1_sum.divide(df1_sum["서울시"], axis = 0)
df1_prop = df1_prop.apply(lambda x: x*100)

- 남자 인구 통계

In [17]:
male_population = df1.xs('남자인구수 (명)', level='성별', axis=1)
male_population["서울시"] = male_population.sum(axis = 1).astype(int)

In [18]:
male_prop = male_population.divide(male_population["서울시"], axis = 0).drop("서울시", axis = 1)
male_prop["비율 합"] = male_prop.sum(axis = 1)

## 서울시 연령대별 남녀 인구수

### 남자 인구수

- 특정 연도 데이터 분리

In [19]:
df2_2014 = df2.xs(2014, level="시점")
df2_2014 = df2_2014.T

## 주민등록인구(내국인 각 세별_구별)

- 생애주기별 인구 분류
    - 아동청소년: 0 ~ 18
    - 청년: 19 ~ 34
    - 중장년: 35 ~ 64
    - 노인: 65 ~

In [20]:
df4_Gen = pd.DataFrame()
ageClassify = [(0, 19, "아동청소년"), (19, 35, "청년"), (35, 65, "중장년"), (65,None, "노인")]

for i in ageClassify:
    start, end, category = i
    temp = df4.iloc[:, start:end]
    temp[category] = temp.sum(axis = 1)
    
    df4_Gen = pd.concat([df4_Gen, temp[[category]]], axis = 1)

## 서울시 자치구 (목적별) CCTV 설치현황

- Multi Columns level 축소
- columns 이름 변경
- index 자치구 -> 서울시 변경

In [115]:
remain = df7["범죄예방 및 수사"][['방범', '어린이 보호구역', '공원·놀이터', '쓰레기 무단투기']]

df7_drop = df7.drop("범죄예방 및 수사", axis = 1)
df7_drop.columns = ["CCTV 총계", "시설안전·화재예방", "교통단속", "교통정보수집·분석", "기타 다른법령"]

df7_prep = pd.concat([df7_drop, remain], axis = 1)
df7_prep = df7_prep.rename(index={"자치구 ":"서울시"})

# 데이터 추출

## 강서구 인구현황(2023년 기준)
- 세대수: 274084
- 남성: 269822
- 여성: 293236
- 외국인: 5768

In [21]:
df5["세대수"] = df5.sum(axis = 1)
household = df5["세대수"]["강서구"]

In [24]:
foreigner = df6["외국인 (명)"]
foreigner["외국인"] = foreigner.sum(axis = 1)

df6_subtotal = pd.concat([df6["한국인 (명)"], foreigner["외국인"]], axis = 1)
df6_subtotal = df6_subtotal[df6_subtotal.index == "강서구"]

df6_subtotal["세대수"] = household
df6_subtotal = df6_subtotal.rename(columns={"남자":"남성", "여자":"여성"})
df6_subtotal[["세대수", "남성", "여성", "외국인"]]
# df6_subtotal.to_excel("./output/2023년 강서구 인구현황.xlsx")

,세대수,남성,여성,외국인
강서구,274084,269822,293236,5768


## 구별 인구현황(2023년 기준)

In [28]:
df6.columns.names = ["국적", "성별"]

temp_male = df6.xs('남자', level='성별', axis=1)
temp_male["남성"] = temp_male.sum(axis = 1)

temp_female = df6.xs('여자', level='성별', axis=1)
temp_female["여성"] = temp_female.sum(axis = 1)

temp_subtotal = pd.concat([temp_male[["남성"]], temp_female[["여성"]]], axis = 1)
temp_subtotal["인구수"] = temp_subtotal.sum(axis = 1)

In [29]:
temp = df5[["세대수"]]
temp_subset = pd.Series(temp[["세대수"]].sum(), name="소계")
temp = temp.append(temp_subset)

df6_status = pd.merge(temp_subtotal, temp, left_index=True, right_index=True, how="left")

df6_status = df6_status[["인구수", "세대수", "남성", "여성"]]
df6_status = df6_status.rename(index={"소계":"서울시"})
df6_status["남녀비율(남성/여성)"] = df6_status["남성"] / df6_status["여성"]

# df6_status.to_excel("./output/2023년 구별 인구현황.xlsx")

### 인구 통계 확인

In [33]:
df6_status.sort_values(by="인구수", ascending=False).head(6)

,인구수,세대수,남성,여성,남녀비율(남성/여성)
서울시,9638799,4469417,4649446,4989353,0.931874
송파구,660025,285927,316981,343044,0.924024
강서구,568826,274084,272338,296488,0.918546
강남구,550282,239775,262991,287291,0.915417
노원구,502925,217904,241099,261826,0.920837
관악구,497883,284578,249026,248857,1.000679


In [34]:
df6_status.sort_values(by="세대수", ascending=False).head(6)

,인구수,세대수,남성,여성,남녀비율(남성/여성)
서울시,9638799,4469417,4649446,4989353,0.931874
송파구,660025,285927,316981,343044,0.924024
관악구,497883,284578,249026,248857,1.000679
강서구,568826,274084,272338,296488,0.918546
강남구,550282,239775,262991,287291,0.915417
노원구,502925,217904,241099,261826,0.920837


In [35]:
df6_status.sort_values(by="남성", ascending=False).head(6)

,인구수,세대수,남성,여성,남녀비율(남성/여성)
서울시,9638799,4469417,4649446,4989353,0.931874
송파구,660025,285927,316981,343044,0.924024
강서구,568826,274084,272338,296488,0.918546
강남구,550282,239775,262991,287291,0.915417
관악구,497883,284578,249026,248857,1.000679
노원구,502925,217904,241099,261826,0.920837


In [36]:
df6_status.sort_values(by="여성", ascending=False).head(6)

,인구수,세대수,남성,여성,남녀비율(남성/여성)
서울시,9638799,4469417,4649446,4989353,0.931874
송파구,660025,285927,316981,343044,0.924024
강서구,568826,274084,272338,296488,0.918546
강남구,550282,239775,262991,287291,0.915417
노원구,502925,217904,241099,261826,0.920837
관악구,497883,284578,249026,248857,1.000679


In [37]:
df6_status.sort_values(by="남녀비율(남성/여성)", ascending=False).head(6)

,인구수,세대수,남성,여성,남녀비율(남성/여성)
금천구,241105,120381,121592,119513,1.017396
관악구,497883,284578,249026,248857,1.000679
구로구,415651,184096,204715,210936,0.970508
영등포구,397800,190737,195493,202307,0.966319
중랑구,387470,188097,189462,198008,0.956840
강동구,463318,203734,226237,237081,0.954260


In [55]:
temp = df6_status.sort_values(by="남녀비율(남성/여성)", ascending=False)
temp_loc = temp.index.get_loc("강서구")
temp.iloc[temp_loc - 2 : temp_loc + 3]

,인구수,세대수,남성,여성,남녀비율(남성/여성)
송파구,660025,285927,316981,343044,0.924024
노원구,502925,217904,241099,261826,0.920837
강서구,568826,274084,272338,296488,0.918546
강남구,550282,239775,262991,287291,0.915417
종로구,150453,72067,71890,78563,0.915062
